In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

from pprint import pprint

import src
from src.reload import deep_reload

In [24]:
'''
Creating a random graph

graph contains n destinations and m chargers

the random graph is completely connected
'''
deep_reload(src)

seed = 125396749
n = 15
m = 85
speeds = [105]
chargers = list(range(10))
scale = (1e6, 1e6)

graph = src.rng.random_completely_connected_graph(
    n, m, speeds = speeds, scale = scale, seed = seed, chargers = chargers,
)

In [10]:
for source, adj in graph._adj.items():
    for target, edge in adj.items():

        if ('station' in source) and ('station' in target):
            
            edge['type'] = 'station_station'

        elif ('station' in source) or ('station' in target):

            edge['type'] = 'destination_station'
            
        else:
            
            edge['type'] = 'destination_destination'

In [11]:
graph.number_of_nodes(), graph.number_of_edges()

(100, 5050)

In [37]:
'''
Running with Dijkstra with default objective
'''
deep_reload(src)

objective = src.dijkstra.Objective(field = 'time')

t0 = time.time()

r = src.dijkstra.dijkstra(
    graph, ['destination_0'], objective = objective, return_paths = True
)

print(f'Executed in {time.time() - t0:.4f} seconds')

Executed in 0.0035 seconds


In [38]:
print('Total Time:\n')
pprint({k: v / 3600 for k, v in r[1].items() if 'station' not in k})

Total Time:

{'destination_0': 0.0,
 'destination_1': 2.419313306788944,
 'destination_10': 15.865801720562095,
 'destination_11': 17.455990274462145,
 'destination_12': 18.390611267492915,
 'destination_13': 19.75785190081337,
 'destination_14': 21.18580806594722,
 'destination_2': 4.886267052326247,
 'destination_3': 4.935066756713083,
 'destination_4': 6.732596645463351,
 'destination_5': 9.465818516275913,
 'destination_6': 9.97586087616289,
 'destination_7': 12.523449524163999,
 'destination_8': 12.86881867295909,
 'destination_9': 14.601186731085448}


In [14]:
'''
Running with Dijkstra with deterministic vehicle objective
'''
deep_reload(src)

objective = src.routing.Vehicle(capacity = 80 * 3.6e6)

t0 = time.time()

r = src.dijkstra.dijkstra(
    graph, ['destination_0'], objective = objective, return_paths = True
)

print(f'Executed in {time.time() - t0:.4f} seconds')

Executed in 0.0787 seconds


In [15]:
print('Total Time:\n')
pprint({k: v['time'] / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nDriving Time:\n')
pprint({k: v['driving_time'] / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nPaths:\n')
pprint({k: v for k, v in r[2].items() if 'station' not in k})

Total Time:

{'destination_0': 0.0,
 'destination_1': 3.329488711570786,
 'destination_10': 1.264614989476647,
 'destination_11': 2.102770479381132,
 'destination_12': 0.9346209930307693,
 'destination_13': 1.3672406333204534,
 'destination_14': 1.9442915941570351,
 'destination_2': 3.3715431972851726,
 'destination_3': 0.048799704386836514,
 'destination_4': 2.248851851178897,
 'destination_5': 3.92068950325288,
 'destination_6': 0.5100423598869759,
 'destination_7': 3.741686048806584,
 'destination_8': 0.34536914879509084,
 'destination_9': 2.2606477189829977}

Driving Time:

{'destination_0': 0.0,
 'destination_1': 2.4281328241292264,
 'destination_10': 1.264614989476647,
 'destination_11': 1.6448868864959576,
 'destination_12': 0.9346209930307693,
 'destination_13': 1.3672406333204534,
 'destination_14': 1.5218147690378636,
 'destination_2': 2.470187309843613,
 'destination_3': 0.048799704386836514,
 'destination_4': 1.8263750260597256,
 'destination_5': 2.7380171401915927,
 'desti

In [17]:
'''
Running with Bellman with default objective
'''
deep_reload(src)

objective = src.bellman.Objective(field = 'time')

t0 = time.time()

r = src.bellman.bellman(graph, ['destination_0'], objective = objective, return_paths = True)

print(f'Executed in {time.time() - t0:.4f} seconds')

Executed in 0.0044 seconds


In [18]:
print('Total Time:\n')
pprint({k: v / 3600 for k, v in r[1].items() if 'station' not in k})

Total Time:

{'destination_0': 0.0,
 'destination_1': 2.419313306788944,
 'destination_10': 1.264614989476647,
 'destination_11': 1.590188553900052,
 'destination_12': 0.9346209930307693,
 'destination_13': 1.3672406333204534,
 'destination_14': 1.4279561651338475,
 'destination_2': 2.4669537455373023,
 'destination_3': 0.048799704386836514,
 'destination_4': 1.7975298887502689,
 'destination_5': 2.7332218708125624,
 'destination_6': 0.5100423598869759,
 'destination_7': 2.5475886480011085,
 'destination_8': 0.34536914879509084,
 'destination_9': 1.732368058126357}


In [19]:
'''
Running with Bellman with BEV vehicle objective
'''
deep_reload(src)

objective = src.routing.Vehicle(capacity = 80 * 3.6e6)

t0 = time.time()

r = src.bellman.bellman(
    graph, ['destination_0'], objective = objective, return_paths = True
)

print(f'Executed in {time.time() - t0:.4f} seconds')

Executed in 0.0123 seconds


In [20]:
print('Total Time:\n')
pprint({k: v['time'] / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nDriving Time:\n')
pprint({k: v['driving_time'] / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nPaths:\n')
pprint({k: v for k, v in r[2].items() if 'station' not in k})

Total Time:

{'destination_0': 0.0,
 'destination_1': 3.329488711570786,
 'destination_10': 1.264614989476647,
 'destination_11': 2.102770479381132,
 'destination_12': 0.9346209930307693,
 'destination_13': 1.3672406333204534,
 'destination_14': 1.9442915941570351,
 'destination_2': 3.3715431972851726,
 'destination_3': 0.048799704386836514,
 'destination_4': 2.248851851178897,
 'destination_5': 3.92068950325288,
 'destination_6': 0.5100423598869759,
 'destination_7': 3.741686048806584,
 'destination_8': 0.34536914879509084,
 'destination_9': 2.2606477189829977}

Driving Time:

{'destination_0': 0.0,
 'destination_1': 2.4281328241292264,
 'destination_10': 1.264614989476647,
 'destination_11': 1.6448868864959576,
 'destination_12': 0.9346209930307693,
 'destination_13': 1.3672406333204534,
 'destination_14': 1.5218147690378636,
 'destination_2': 2.470187309843613,
 'destination_3': 0.048799704386836514,
 'destination_4': 1.8263750260597256,
 'destination_5': 2.7380171401915927,
 'desti

In [21]:
'''
Running with Bellman with ICEV vehicle objective
'''
deep_reload(src)

objective = src.routing.Vehicle(
    capacity = 528 * 3.6e6,
    efficiency = 550 * 5,
    charge_rate = 14.15e6,
    charge_time_penalty = 300,
)

t0 = time.time()

r = src.bellman.bellman(
    graph, ['destination_0'], objective = objective, return_paths = True
)

print(f'Executed in {time.time() - t0:.4f} seconds')

Executed in 0.0153 seconds


In [22]:
print('Total Time:\n')
pprint({k: v['time'] / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nDriving Time:\n')
pprint({k: v['driving_time'] / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nPaths:\n')
pprint({k: v for k, v in r[2].items() if 'station' not in k})

Total Time:

{'destination_0': 0.0,
 'destination_1': 2.528332822970335,
 'destination_10': 1.264614989476647,
 'destination_11': 1.590188553900052,
 'destination_12': 0.9346209930307693,
 'destination_13': 1.3672406333204534,
 'destination_14': 1.4279561651338475,
 'destination_2': 2.5750250868361086,
 'destination_3': 0.048799704386836514,
 'destination_4': 1.7975298887502689,
 'destination_5': 2.8492065342257633,
 'destination_6': 0.5100423598869759,
 'destination_7': 2.6489220811182093,
 'destination_8': 0.34536914879509084,
 'destination_9': 1.732368058126357}

Driving Time:

{'destination_0': 0.0,
 'destination_1': 2.4212072542306164,
 'destination_10': 1.264614989476647,
 'destination_11': 1.590188553900052,
 'destination_12': 0.9346209930307693,
 'destination_13': 1.3672406333204534,
 'destination_14': 1.4279561651338475,
 'destination_2': 2.468332861799405,
 'destination_3': 0.048799704386836514,
 'destination_4': 1.7975298887502689,
 'destination_5': 2.7457196183534522,
 'des

In [25]:
'''
Adding station objects
'''
deep_reload(src)

vehicle = src.routing.StochasticVehicle(
    capacity = 80 * 3.6e6,
    max_charge_start_soc = .4,
)

for source, node in graph._node.items():

    if node['type'] == 'station':

        node['station'] = src.routing.StochasticStation(
            vehicle, chargers = node['n_dcfc']
        )

In [31]:
'''
Running with Bellman with BEV stochastic vehicle objective
'''
deep_reload(src)

t0 = time.time()

r = src.bellman.bellman(
    graph, ['destination_0'], objective = vehicle, return_paths = True
)


print(f'Executed in {time.time() - t0:.4f} seconds')

Executed in 1.2987 seconds


In [32]:
print('Total Time:\n')
pprint({k: v['time'].mean() / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nDriving Time:\n')
pprint({k: v['driving_time'].mean() / 3600 for k, v in r[1].items() if 'station' not in k})
print('\nPaths:\n')
pprint({k: v for k, v in r[2].items() if 'station' not in k})

Total Time:

{'destination_0': 0.0,
 'destination_1': 3.2969017192423196,
 'destination_10': 1.2646149894766467,
 'destination_11': 2.46073660207157,
 'destination_12': 0.9346209930307695,
 'destination_13': 1.3672406333204536,
 'destination_14': 2.384902451559524,
 'destination_2': 3.332668941053535,
 'destination_3': 0.04879970438683653,
 'destination_4': 2.6187077036622624,
 'destination_5': 4.321046022501706,
 'destination_6': 0.5100423598869761,
 'destination_7': 4.121681511108887,
 'destination_8': 0.3453691487950909,
 'destination_9': 2.701675875558017}

Driving Time:

{'destination_0': 0.0,
 'destination_1': 2.4325656399881885,
 'destination_10': 1.2646149894766467,
 'destination_11': 1.59640052281744,
 'destination_12': 0.9346209930307695,
 'destination_13': 1.3672406333204536,
 'destination_14': 1.4537726387849168,
 'destination_2': 2.4683328617994045,
 'destination_3': 0.04879970438683653,
 'destination_4': 1.933145830721578,
 'destination_5': 2.7964000390561856,
 'destinati